In [1]:
import numpy as np
from Tools.data_utils import load_CIFAR10

In [2]:
cifar10_dir = 'C:/Users/rainstar/Jupyter Folder/cs231n/cifar-10-batches-py/'

try:
    del X_train,Y_train
    del X_test,Y_test
    print('Clear previously loaded data')
except:
    pass

X_train,Y_train,X_test,Y_test = load_CIFAR10(cifar10_dir)

print('Training data shape: ',X_train.shape)
print('Training labels shape: ',Y_train.shape)
print('Test data shape: ',X_test.shape)
print('Test labels shape: ',Y_test.shape)

Training data shape:  (50000, 3072)
Training labels shape:  (50000,)
Test data shape:  (10000, 3072)
Test labels shape:  (10000,)


In [3]:
num_train = 49000
num_validation = 1000
num_test = 1000
num_dev =  500

mask = range(num_train,num_train+num_validation)
X_val = X_train[mask]
Y_val = Y_train[mask].astype('int')

mask = range(num_train)
X_train = X_train[mask]
Y_train = Y_train[mask].astype('int')

mask = range(num_test)
X_test = X_test[mask]
Y_test = Y_test[mask].astype('int')

mask = np.random.choice(num_train,num_dev,replace=True)
X_dev = X_train[mask]
Y_dev = Y_train[mask].astype('int')

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', Y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', Y_val.shape)
print('Test data shape: ', Y_test.shape)
print('Test labels shape: ', Y_test.shape)

Train data shape:  (49000, 3072)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3072)
Validation labels shape:  (1000,)
Test data shape:  (1000,)
Test labels shape:  (1000,)


In [4]:
# normalize
mean_image = np.mean(X_train,axis=0)

X_train -= mean_image
X_val -= mean_image
X_test -= mean_image
X_dev -= mean_image

# append bias
X_train = np.hstack([X_train,np.ones((X_train.shape[0],1))])
X_val = np.hstack([X_val,np.ones((X_val.shape[0],1))])
X_test = np.hstack([X_test,np.ones((X_test.shape[0],1))])
X_dev = np.hstack([X_dev,np.ones((X_dev.shape[0],1))])

print('X_train: ',X_train.shape)
print('X_val: ',X_val.shape)
print('X_test: ',X_test.shape)
print('X_dev: ',X_dev.shape)

X_train:  (49000, 3073)
X_val:  (1000, 3073)
X_test:  (1000, 3073)
X_dev:  (500, 3073)


In [5]:
from Tools.classifiers.softmax import *

W = np.random.randn(3073,10) * 0.0001

loss1,grad1 = softmax_loss_naive(W,X_dev,Y_dev,0.0)
print('loss: ',loss1)

loss2,grad2 = softmax_loss_vectorized(W,X_dev,Y_dev,0.0)
print('loss: ',loss2)

loss:  2.3901293995394086
loss:  2.3901293995394086


In [9]:
from Tools.classifiers.linear_classifier import *

best_val = -1
best_softmax = None

learning_rates = [5e-8,1e-7,5e-7]
regularizations = [2.5e4,5e4]

for lr in learning_rates:
    for reg in regularizations:
        softmax = LinearSoftmax()
        softmax.train(X_train,Y_train,learning_rate=lr,
                     regu=reg,num_iters=1500,batch_size=200,
                     verbose=False)
        train_acc = np.mean(Y_train==softmax.predict(X_train))
        val_acc = np.mean(Y_val==softmax.predict(X_val))
        print("lr {} regu {} : train_acc{} val_acc{}".format(lr,reg,
                                                            train_acc,
                                                            val_acc))
        if val_acc>best_val:
            best_val = val_acc
            best_softmax = softmax

lr 5e-08 regu 25000.0 : train_acc0.31587755102040815 val_acc0.324
lr 5e-08 regu 50000.0 : train_acc0.3187755102040816 val_acc0.329
lr 1e-07 regu 25000.0 : train_acc0.3451836734693878 val_acc0.361
lr 1e-07 regu 50000.0 : train_acc0.32026530612244897 val_acc0.336
lr 5e-07 regu 25000.0 : train_acc0.340265306122449 val_acc0.358
lr 5e-07 regu 50000.0 : train_acc0.3146734693877551 val_acc0.329


In [10]:
Y_test_pred = best_softmax.predict(X_test)
test_acc = np.mean(Y_test == Y_test_pred)
print("softmax final accuracy on test set: ",test_acc)

softmax final accuracy on test set:  0.355
